In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.getcwd()

'/data/aqvs/notebooks'

In [3]:
import sys, glob, shutil
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/data/aqvs'

#### Adding "src/networks" folder in path, to enable in-line imports for the network files using importlib

In [4]:
import os, sys
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
sys.path.append(os.path.abspath('./src/networks'))

#### To handel OOM errors

In [5]:
import tensorflow as tf
from keras import backend as K
import keras.backend.tensorflow_backend as ktf
def get_session():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction= 0.8,
                                allow_growth=True)
    return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
ktf.set_session(get_session())

Using TensorFlow backend.


In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import SGD, Adam, RMSprop

In [7]:
from config import objd as config
from src.utils.preprocessing import parse_annotation

In [8]:
from src.training.datagenerator import ObjectDataGenerator as DataGenerator

In [9]:
from src.networks.yolo import Yolo
from src.networks.feature_extractors import get_feature_extractor

In [10]:
from src.training.keras_callbacks import get_callbacks
from src.evaluation.evaluate import evaluate

### Parse Config

In [11]:
backend = config['model']['backend']
input_size = config['model']['input_size']
labels = config['model']['labels']
max_box_per_image = config['model']['max_box_per_image']
anchors = config['model']['anchors']

train_times = config['train']['train_times']

nb_epochs = config['train']['nb_epochs']
learning_rate = config['train']['learning_rate'] 
batch_size = config['train']['batch_size']
warmup_epochs = config['train']['warmup_epochs']

object_scale = config['train']['object_scale']
no_object_scale = config['train']['no_object_scale']
coord_scale = config['train']['coord_scale']
class_scale = config['train']['class_scale']

labels   = list(labels)
nb_class = len(labels)
nb_box   = len(anchors)//2
class_wt = np.ones(nb_class, dtype='float32')

valid_times = 1

### Parse Annotations

In [12]:
# parse annotations of the training set
train_imgs, train_labels = parse_annotation(config['train']['train_annot_folder'], 
                                            config['train']['train_image_folder'], 
                                            config['model']['labels'])
train_labels, train_imgs 

({'crack': 103, 'wrinkle': 56},
 [{'object': [{'name': 'crack',
     'xmin': 1461,
     'ymin': 2044,
     'xmax': 2386,
     'ymax': 2707},
    {'name': 'wrinkle',
     'xmin': 1108,
     'ymin': 1227,
     'xmax': 1508,
     'ymax': 1674}],
   'filename': 'data/images/IMG20180905143750.jpg',
   'width': 4160,
   'height': 3120},
  {'object': [{'name': 'crack',
     'xmin': 588,
     'ymin': 1478,
     'xmax': 859,
     'ymax': 2111},
    {'name': 'wrinkle',
     'xmin': 1422,
     'ymin': 1113,
     'xmax': 1946,
     'ymax': 1437}],
   'filename': 'data/images/IMG20180905143945.jpg',
   'width': 3120,
   'height': 4160},
  {'object': [{'name': 'crack',
     'xmin': 530,
     'ymin': 1386,
     'xmax': 1088,
     'ymax': 1740}],
   'filename': 'data/images/IMG20180905144032.jpg',
   'width': 3120,
   'height': 4160},
  {'object': [{'name': 'crack',
     'xmin': 755,
     'ymin': 1378,
     'xmax': 930,
     'ymax': 1524}],
   'filename': 'data/images/IMG20180905144103.jpg',
   'width

### Prepare training and validation data

In [13]:
train_valid_split = int(0.8*len(train_imgs))
np.random.shuffle(train_imgs)
valid_imgs = train_imgs[train_valid_split:]
train_imgs = train_imgs[:train_valid_split]

### Import base feature extractor

In [14]:
feature_extractor, grid_h, grid_w = get_feature_extractor(backend, input_size)

Instructions for updating:
Colocations handled automatically by placer.
Output shape for the feature extractor is: (9, 9)


### Initialise DataGenerator

In [15]:
generator_config = {
    'IMAGE_H'         : input_size,
    'IMAGE_W'         : input_size,
    'GRID_H'          : grid_h,
    'GRID_W'          : grid_w,
    'BOX'             : nb_box,
    'LABELS'          : labels,
    'CLASS'           : len(labels),
    'ANCHORS'         : anchors,
    'BATCH_SIZE'      : batch_size,
    'TRUE_BOX_BUFFER' : max_box_per_image,
}

In [16]:
train_generator = DataGenerator(train_imgs, generator_config, norm=feature_extractor.normalize)
valid_generator = DataGenerator(valid_imgs, generator_config, norm=feature_extractor.normalize, jitter=False)

### Testing datagenerator (optional)

In [ ]:
[x_batch, b_batch], y_batch = train_generator.__getitem__(1)

In [ ]:
x_batch.shape, b_batch.shape, y_batch.shape

### Import and compile object detection model

In [17]:
warmup_batches  = warmup_epochs * (train_times*len(train_generator) + valid_times*len(valid_generator))

In [18]:
yolo = Yolo(nb_class, batch_size, anchors, object_scale, max_box_per_image, no_object_scale, coord_scale, 
            class_scale, nb_box, grid_h, grid_w, warmup_batches)

In [19]:
model = yolo.build(input_shape=(input_size, input_size,3), feature_extractor=feature_extractor, labels=labels)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 9, 9, 1024)   15739760    input_2[0][0]                    
__________________________________________________________________________________________________
DetectionLayer (Conv2D)         (None, 9, 9, 49)     50225       model_1[1][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 9, 9, 7, 7)   0           DetectionLayer[0][0]             
__________________________________________________________________________________________________
input_3 (I

In [20]:
custom_loss = yolo.loss

In [21]:
optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [22]:
model.compile(loss=custom_loss, optimizer=optimizer)

### Define callbacks

In [23]:
callbacks_list = get_callbacks(model)

### Start model training

In [ ]:
model.fit_generator(generator = train_generator,
                    steps_per_epoch  = len(train_generator) * train_times,
                    epochs           = warmup_epochs + nb_epochs,
                    validation_data  = valid_generator,
                    validation_steps = len(valid_generator) * valid_times,
                    callbacks        = callbacks_list,
                    workers          = 3,
                    max_queue_size   = 8)

Epoch 1/18
48/48 [==============================] - 389s 8s/step - loss: 10.3269 - val_loss: 10.2499

Epoch 00001: saving model to ./data/yolo_t1/checkpoint-01-10.25.h5
Epoch 2/18
48/48 [==============================] - 370s 8s/step - loss: 10.0756 - val_loss: 10.1846

Epoch 00002: saving model to ./data/yolo_t1/checkpoint-02-10.18.h5
Epoch 3/18
48/48 [==============================] - 359s 7s/step - loss: 10.0475 - val_loss: 10.1332

Epoch 00003: saving model to ./data/yolo_t1/checkpoint-03-10.13.h5
Epoch 4/18
48/48 [==============================] - 395s 8s/step - loss: 0.4678 - val_loss: 1.9367

Epoch 00004: saving model to ./data/yolo_t1/checkpoint-04-1.94.h5
Epoch 5/18
48/48 [==============================] - 360s 7s/step - loss: 2.4489 - val_loss: 0.9497

Epoch 00005: saving model to ./data/yolo_t1/checkpoint-05-0.95.h5
Epoch 6/18
48/48 [==============================] - 397s 8s/step - loss: 0.4420 - val_loss: 0.4381

Epoch 00006: saving model to ./data/yolo_t1/checkpoint-06-0.4

## Prediction

In [ ]:
image_path = "data/images/IMG20180905143750.jpg"
image = cv2.imread(image_path)
plt.imshow(image)

In [ ]:
#! /usr/bin/env python
import cv2
import numpy as np
from keras.models import load_model

from config import infer as config
from src.utils.utils import decode_netout, draw_boxes
from src.networks.feature_extractors import get_feature_extractor

In [ ]:
model_path = config["model_path"]
backend = config['backend']
input_size = config["input_size"]
max_box_per_image = config["max_box_per_image"]
anchors = config["anchors"]
labels = config["labels"]        
nb_class = len(labels)
model = load_model(model_path, compile=False)
feature_extractor, grid_h, grid_w = get_feature_extractor(backend, input_size)
normalize = feature_extractor.normalize

In [ ]:
image_h, image_w, _ = image.shape
image = cv2.resize(image, (input_size, input_size))
image = normalize(image)
    
input_image = image[:,:,::-1]
input_image = np.expand_dims(input_image, 0)
dummy_array = np.zeros((1,1,1,1,max_box_per_image,4))

In [ ]:
netout = model.predict([input_image, dummy_array])[0]    
netout

In [ ]:
boxes  = decode_netout(netout, anchors, nb_class)
boxes

In [ ]:
image = draw_boxes(image, boxes, labels)
plt.imshow(image)

### Evaluate model

In [ ]:
average_precisions = evaluate(valid_generator)

In [ ]:
# print evaluation
for label, average_precision in average_precisions.items():
    print(labels[label], '{:.4f}'.format(average_precision))
print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))